In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.util import Dynamic
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

# set pandas dataframe float precision 
pd.set_option('display.precision',2)


In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
import utils as u
import hv_utils as  hvu


In [ ]:
mro = u.get_mro

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom']),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


In [ ]:
H,W = 500,500

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
data_dir = Path.home()/'data'
fpath_sa = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
)
fpath_ea = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
)
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

---
## Back to FLDAS
Modified: Jun 25, 2019

Combining holoviews objects with Bokeh models for custome interactive visualization


In [ ]:
# Set extra style opts (in addition to default from above)
W_IMG = 500; H_IMG = 500
W_PLOT = 300; H_PLOT = 300

In [ ]:
scatter_opts = dict(width=W_PLOT, height=H_PLOT,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W_PLOT, height=H_PLOT,
                  framewise=True)
img_opts = dict(width=W_IMG, height=H_IMG,
                axiswise=True, 
                framewise=False,
                tools=['hover', 'tap'],
                colorbar=True
               )
wmts_opts = dict(width=W_IMG, height=H_IMG)

tbl_opts = dict(width = W_PLOT)

# datashader opts
ds_opts = dict(width=W_IMG, height=H_IMG,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

---
## Clean BoxEdit from Scratch
Modified: Jun 26, 2019

- Incoporating HoloMap and DynamicMap objects
    - [src](http://holoviews.org/user_guide/Live_Data.html)

In [ ]:
xrd=xrd_ea
varname=varnames[7]
t = '2019-04-10'
time_values = xrd.get_index('time')
# time_values = pd.date_range('2019-04-01', '2019-04-30', freq='D')

data = xrd[varname]#.sel(time=t)
hvd = gv.Dataset(data, ['X','Y','time'], varname, crs=ccrs.PlateCarree())

def get_timg(time):
    data = xrd[varname].sel(time=time)
    return gv.Image(data, ['X','Y'], varname, crs=ccrs.PlateCarree())
dmap_timg = datashade(hv.DynamicMap(get_timg, kdims='time'),
                      **ds_opts)
# dmap_timg = hv.DynamicMap(lambda time: datashade(get_timg(time), **ds_opts, dynamic=False), kdims='time')

In [ ]:
print(dmap_timg)
dmap_timg.redim.values(time=time_values)

In [ ]:
print(dmap_timg)
dmap_timg.redim.values(time=time_values)

In [ ]:
pn.panel(dmap_timg.redim.values(time=time_values))

In [ ]:
# dmap_realized = dmap_timg.redim.values(time=time_values)
dmap_realized = dmap_timg.redim.values(time=[dt.datetime(2019,4,t) for t in range(1,31)])
dmap_realized.dimension_values('time')
pn.panel(dmap_realized)

In [ ]:
# Set BoxEdit stream
polys = gv.Polygons([], crs=ccrs.PlateCarree())
box_stream = streams.BoxEdit(source=polys)
dmap = dmap_timg * polys

In [ ]:
def get_empty_tplot():
    # Set empty tplot and vlines
    dummy_df = pd.DataFrame({'time': time_values, 
                             varname:np.zeros(len(time_values))})
    empty_tplot= hv.Curve(dummy_df, 'time', varname).opts(line_alpha=0.)
    return empty_tplot

def get_hvmap_vlines(vline_opts={}):
    # Set empty tplot and vlines
    empty_tplot= get_empty_tplot()
    vlines = hv.HoloMap({t: hv.VLine(t).opts(**vline_opts) for t in time_values},
                        kdims='time')
#     return empty_tplot * vlines
    return vlines

empty_tplot = get_empty_tplot()
# vlines = get_hvmap_vlines(dict(color='green')) 
vlines = hv.DynamicMap(lambda time: empty_tplot * hv.VLine(time), kdims='time') 
vlines.opts(opts.Curve(bgcolor=(0,0,0,0)))
# vlines = hv.DynamicMap(lambda time: empty_tplot * hv.VLine(time) *dmap_roi_curves, kdims='time') 

print(vlines)

In [ ]:
if any( not (mro(t)[0] == dt.datetime) for t in time_values):
    t2 = list(map(lambda t: t.to_pydatetime(),time_values))
    print('converted timevalues to python datetime object')
    
# t2

In [ ]:
mro(t0)[0],mro(t0.to_pydatetime())
d0 = t0.to_pydatetime()
mro(d0),mro(d0)[0] == dt.datetime

In [ ]:
dmap_realized = dmap_timg.redim.values(time=[dt.datetime(2019,4,t) for t in range(1,31)])
v = vlines.redim.values(time=[dt.datetime(2019,4,t) for t in range(1,31)])
polys*dmap_realized + v

In [ ]:
# p = pn.panel(dmap + vlines*empty_tplot)

Success!!!

In [ ]:
def roi_curves(data):
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: empty_tplot})
    curves = {}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0,x1,y0,y1) in enumerate(data):
        selection = hv.Dataset( xrd[varname].sel(X=slice(x0,x1), Y=slice(y0,y1)), kdims=['X','Y','time'])
        curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', varname)
    return hv.NdOverlay(curves, label='roi_curves')

dmap_roi_curves = hv.DynamicMap(roi_curves, streams=[box_stream])

In [ ]:
print(dmap_roi_curves)
v*dmap_roi_curves 

In [ ]:
# will it finally work?
layout=basemap*dmap_realized *polys + v*dmap_roi_curves

In [ ]:
# layout.opts(
#     opts.Curve(width=400,framewise=True),
#     opts.VLine(color='black'),
#     opts.Polygons(fill_alpha=0.2, line_color='white', fill_color='orange'), 
# )

In [ ]:
pn.panel(layout).servable()

woohoo!!!!! I can go to bed now:D

In [ ]:
test_rois = {'x0': [28.484264530634945, 23.75232112846949], 
             'y0': [5.161857571360065, -4.790934173611567], 
             'x1': [39.80125756910407, 35.76445237820839], 
             'y1': [16.341892860897076, 10.938420208725127]}
# test_rois
box_stream.event(data=test_rois)
# box_stream.event(data={})

In [ ]:
trange

In [ ]:
trange = list(map(pd.Timestamp, hvd_ea.range('time')))
class Time(param.Parameterized):
    
    time= param.Date(trange[0], bounds=trange)
    

In [ ]:
# time stream
tparam = Time()
tparam
stream = streams.Params(tparam)
twidget =pn.panel(tparam.param
         , widgets={'time': pn.widgets.DateSlider})

In [ ]:
## better way

from holoviews.streams import Stream, param

# Trial1: use datetime as time input/key
Time = Stream.define('Time', t=0.0)
Time = streams.Stream.define('Time', time=param.Date(trange[0], bounds=trange, doc='a time parameter'))


In [ ]:
tparam = Time()
twidget =pn.panel(tparam.param
         , widgets={'time': pn.widgets.DateSlider})
print(tparam)
hv.help(Time)

In [ ]:
#Take 2: use time index (integer btw 0 and 30)
Tidx = streams.Stream.define(
    'Tidx', time=param.Integer(0, bounds=(0,30)))
tidx_stream = Tidx()
tidx_stream
tidx_widget = pn.panel(tidx_stream.param)
# display(tidx_widget)

dmap_tidx_debug = hv.DynamicMap(hvu.get_debug_div,
                                streams=[tidx_stream])
# display(tidx_widget)
pn.Row(tidx_widget, dmap_tidx_debug).servable()

In [ ]:
u.nprint(tparam, twidget.objects)

In [ ]:
dmap_debug = hv.DynamicMap( hvu.get_debug_div, streams=[tparam, box_stream])

In [ ]:
dmap_vline = hv.DynamicMap(lambda time: hv.VLine(time), streams=[tparam])

In [ ]:
dmap_img = hv.DynamicMap(get_timg, streams=[tparam])
dmap = dmap_img * polys



In [ ]:
twidget = pn.panel(tparam.param, widgets={'time': pn.widgets.DateSlider})

pn.Row( twidget,
       (dmap + dmap_debug+dmap_vline).cols(1)
    ).servable()

In [ ]:
tparam.event(time=pd.Timestamp('2019-04-05'))

In [ ]:
# pointxy = streams.PointerXY(source=
# debug = hv.DynamicMap(hvu.get_debug_div,

In [ ]:
# pn.panel(tparam.param)

In [ ]:
# tparam


In [ ]:
# tparam.trigger(tparam)

In [ ]:
# len(tparam.subscribers), tparam.subscribers